# Code for learning a differentiable weight mask on a transformer

## First, Loading a Model
GPT-2 Medium for now, in Huggingface Transformer library

In [7]:
%load_ext autoreload
%autoreload 2
import os
from cb_utils.models import load_gpt2_weights, load_demo_gpt2, tokenizer
from torch.optim import AdamW
import torch
import pickle
import datasets
from tqdm import tqdm_notebook as tqdm
from itertools import cycle
# from eval import evaluate_model
import plotly.express as px
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
model = load_demo_gpt2(means=None)

In [5]:
conceptnet_data = pd.read_csv('datasets/conceptnet-assertions-5.7.0/assertions.csv', header=None)
# Filter the data to include only rows with single-token tail entities
# filtered_data = conceptnet_data[conceptnet_data[2].apply(lambda x: len(x.split()) == 1)]

ParserError: Error tokenizing data. C error: Expected 7 fields in line 868, saw 9
